# Imports

In [3]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
from VAE1D import *

In [5]:
from IPython.core.debugger import set_trace

# Build model and loss function 
Initially designed for 2D input images, modified for 1D time-series data.
Based on this paper: https://arxiv.org/abs/1807.01349

In [6]:
# The hydraulic system has 14 sensors from which to pull data
n_channels = 14
# The data has been resized to 512, although it represents 1 min for each cycle
size = 512
# Latent space is restricted to be about 1/170th of the input dims, similar to 2D case
n_latent = 50

In [7]:
model = VAE1D(size, n_channels, n_latent)
model

VAE1D(
  (encoder): Sequential(
    (input-conv): Conv1d(14, 16, kernel_size=(4,), stride=(2,), padding=(1,))
    (input-relu): ReLU(inplace)
    (pyramid_16-32_conv): Conv1d(16, 32, kernel_size=(4,), stride=(2,), padding=(1,))
    (pyramid_32_batchnorm): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (pyramid_32_relu): ReLU(inplace)
    (pyramid_32-64_conv): Conv1d(32, 64, kernel_size=(4,), stride=(2,), padding=(1,))
    (pyramid_64_batchnorm): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (pyramid_64_relu): ReLU(inplace)
    (pyramid_64-128_conv): Conv1d(64, 128, kernel_size=(4,), stride=(2,), padding=(1,))
    (pyramid_128_batchnorm): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (pyramid_128_relu): ReLU(inplace)
    (pyramid_128-256_conv): Conv1d(128, 256, kernel_size=(4,), stride=(2,), padding=(1,))
    (pyramid_256_batchnorm): BatchNorm1d(256, eps=1e-05, momentum=0.1,

In [8]:
beta = 10  # KL term relative weight
criterion = VAE1DLoss(beta)

# Load hydraulic test data
From this dataset: https://archive.ics.uci.edu/ml/datasets/Condition+monitoring+of+hydraulic+systems#

In [9]:
data_path = Path('data/hydraulic')
train_dl, val_dl, test_dl = load_datasets(data_path)

In [10]:
print(len(train_dl), len(val_dl), len(test_dl))

38 10 720


# Prepare for training

In [11]:
desc = 'hydraulic'
log_freq = 10 # batches

# Training parameters
epochs = 100
lr = 1e-2                # learning rate
lr_decay = 0.1           # lr decay factor
schedule = [25, 50, 75]  # decrease lr at these epochs
batch_size = 32
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Checkpoint to resume from (default None)
load_path = None

# Checkpoint save location
save_path = Path(f"models/{date.today().strftime('%y%m%d')}-{desc}/")
if save_path.is_dir():
    print(f"Folder {save_path} already exists")
else:
    os.mkdir(save_path)
save_path

PosixPath('models/190129-hydraulic')

In [18]:
# Load checkpoint if any
if load_path is not None:
    checkpoint = torch.load(load_path, map_location=device)
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    print("Checkpoint loaded")
    print(f"Validation loss: {checkpoint['val_loss']}")
    print(f"Epoch: {checkpoint['epoch']}")

In [12]:
# Load optimizer and scheduler
optimizer = torch.optim.Adam(params=model.parameters(), lr=lr)
# scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, schedule, lr_decay)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 10)

In [13]:
# Move to GPU
model = model.to(device)
criterion = criterion.to(device)

# Train the model

In [15]:
def train_VAE1D(dl):
    """
    Execute the training loop
    """
    loss_tracker = AvgTracker()
    kl_tracker = AvgTracker()
    logp_tracker = AvgTracker()
    timer = StopWatch()
    
    for i, (X, _) in enumerate(tqdm(dl)):
        
        X = X.to(device)
        timer.lap()  # load time
        
        # Generate transient and compute loss
        X_hat, mu, logvar = model(X)
        loss, loss_desc = criterion(X_hat, X, mu, logvar)
        timer.lap()  # gen time
        
        loss_tracker.update(loss.item())
        kl_tracker.update(loss_desc['KL'].item())
        logp_tracker.update(loss_desc['logp'].item())
        
        if model.training:
            # Update weights
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            timer.lap()  # backprop time
        
        if (i + 1) % log_freq == 0:
            # Print progress
            print(f'Epoch: {epoch + 1} ({i + 1}/{len(dl)})')
            print(f'\tData load time: {timer.elapsed[0]:.3f} sec')
            print(f'\tGeneration time: {timer.elapsed[1]:.3f} sec')
            if model.training:
                print(f'\tBackprop time: {timer.elapsed[2]:.3f} sec')
            print(f'\tLog probability: {logp_tracker.val:.4f} '
                  f'(avg {logp_tracker.avg:.4f})')
            print(f'\tKL: {kl_tracker.val:.4f} (avg {kl_tracker.avg:.4f})')
            print(f'\tLoss: {loss_tracker.val:.4f} (avg {loss_tracker.avg:.4f})')

    return loss_tracker.avg, kl_tracker.avg, logp_tracker.avg

In [16]:
# Main loop
best_loss = np.inf
for epoch in range(epochs):

    model.train()
    scheduler.step()
    train_loss, train_kl, train_logp = train_VAE1D(train_dl)
    
    model.eval()
    with torch.no_grad():
        val_loss, val_kl, val_logp = train_VAE1D(val_dl)

    # Report training progress to user
    if val_loss < best_loss:
        print('Saving checkpoint..')
        best_loss = val_loss
        save_dict = {'epoch': epoch + 1,
                     'state_dict': model.state_dict(),
                     'val_loss': val_loss,
                     'optimizer': optimizer.state_dict()}
        path = save_path / f'best_model-{n_latent}.pt'
        torch.save(save_dict, path)
    print(f'Lowest validation loss: {best_loss:.4f}')

 32%|███▏      | 12/38 [00:00<00:02, 12.40it/s]

Epoch: 1 (10/38)
	Data load time: 0.139 sec
	Generation time: 0.162 sec
	Backprop time: 0.189 sec
	Log probability: -861.2078 (avg -1047.5210)
	KL: 222.1888 (avg 136883.9179)
	Loss: 3083.0959 (avg 1369886.7162)


 63%|██████▎   | 24/38 [00:01<00:00, 20.14it/s]

Epoch: 1 (20/38)
	Data load time: 0.139 sec
	Generation time: 0.162 sec
	Backprop time: 0.189 sec
	Log probability: -682.0975 (avg -912.2179)
	KL: 161.0796 (avg 68535.5555)
	Loss: 2292.8936 (avg 686267.7805)


 87%|████████▋ | 33/38 [00:01<00:00, 23.44it/s]

...

 34%|███▍      | 13/38 [00:00<00:01, 15.01it/s]

Epoch: 230 (10/38)
	Data load time: 0.117 sec
	Generation time: 0.127 sec
	Backprop time: 0.157 sec
	Log probability: -14.3877 (avg -9.7148)
	KL: 6.2164 (avg 4.0405)
	Loss: 76.5522 (avg 50.1203)


 58%|█████▊    | 22/38 [00:00<00:00, 20.68it/s]


Epoch: 230 (20/38)
	Data load time: 0.117 sec
	Generation time: 0.127 sec
	Backprop time: 0.157 sec
	Log probability: -16.4101 (avg -9.6806)
	KL: 3.3908 (avg 3.8471)
	Loss: 50.3178 (avg 48.1513)


KeyboardInterrupt: 

## Sanity check

Confirm model output is as expected for a test sample.

In [ ]:
for X, y in test_dl:
    break
X.shape

In [ ]:
X[:, :, :4]

In [ ]:
X = X.to(device)

In [ ]:
X.shape

In [ ]:
y

In [ ]:
X_hat, mu, logvar = model(X)

In [ ]:
X

In [ ]:
X_hat